# Objection detection of ping-pong tables using Detectron 2 

* Aim: Predict bounding box around ping-pong tables on map tiles.
* Input: Map tiles, width: 256 pixels, height 256 pixels

* We'll train a ping-pong table object detection model from an existing model pre-trained on COCO dataset, available in detectron2's model zoo.

* Candidate (Faster R-CNN) Pretrained Models:

  * ResNet50_3x
  * ResNet101_3x
  * X101-FPN_3x 

# Check to which GPU I'm assigned & make sure I am using high RAM

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Check number of GPUs
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()


# Install detectron2

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html


# Import libraries and utilities

In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import datetime
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Train on a custom dataset

## Mount to google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# Define ping-pong tables IDs for map tiles




#### Get tile's X & Y coordinates of each map tile in  dataset


In [ ]:
import glob
img_dir_json = "/gdrive/MyDrive/AnasPingPong/data/detection/positive_w_bbox"

list_files_pos = [os.path.basename(x) for x in glob.glob(f"{img_dir_json}/*.json")]
print(f"All tiles:  {len(list_files_pos)}")
pos_tiles_x = [
    int(os.path.basename(f).split("_")[1])
    for f in list_files_pos
]

pos_tiles_y = [
    int(os.path.basename(f).split("_")[2].split(".")[0])
    for f in list_files_pos
]

#### Create ping-pong table's ID by clustering neighbouring map tiles

* If the map tile's X & Y coordinates are close to previously defined tables => tile belongs to the same table => assign same ID

In [ ]:
ids = list()
id_unq_count = 0

for i, (x, y) in enumerate(zip(pos_tiles_x, pos_tiles_y)):
    diff_prev_x = x - np.array(pos_tiles_x[:i]) # empty for first element
    diff_prev_y = y - np.array(pos_tiles_y[:i]) 
    diffs_prev_xy = np.vstack((diff_prev_x, diff_prev_y)).T
    ppt_exists_mask = np.all(diffs_prev_xy>=-1, axis=1) & np.all(diffs_prev_xy<=1, axis=1)
    if any(ppt_exists_mask):
      i_true = [k for k, b in enumerate(ppt_exists_mask) if b]
      ids.append(ids[i_true[0]])
    else:
      ids.append(id_unq_count)
      id_unq_count += 1 

print(f"Unique tables: {len(set(ids))}")

# Create train & validation list of files

* **Important note**: Make sure that all tiles with a given ID (ie corresponding to same table) are assigned to train or validation but not both, to avoid information leakage.


In [ ]:
SEED = 43
val_size_fraction = 0.2
n_pos_all = len(list_files_pos)
n_pos_val = round(n_pos_all * val_size_fraction)
indices_id = list(range(n_pos_all))

diff_exp_vs_curr = 100 # initialize it high

# Reshuffle if we end up with a lot more tables than wanted 
while diff_exp_vs_curr > 10:
  # Create seed to be used for both POSITIVE and NEGATIVE dataset creation
  # SEED = random.randrange(sys.maxsize) 

  indices_id_shuffled = indices_id.copy()
  # random.shuffle(indices_id_shuffled)
  random.Random(SEED).shuffle(indices_id_shuffled)
  i_pos_val = []
  i_pos_trn = []
  for i in indices_id_shuffled:
    if i in i_pos_val:
      continue
    id = ids[i]
    ii_id = list(np.where(np.array(ids) == id)[0]) # find all indices corresponding to this table
    i_pos_val.extend(ii_id) # doesn't add new list -> extends the existing one
    # remove added ones from indices_id_shuffled so that we dont' find again the same table
    # indices_id_shuffled = [x for x in indices_id_shuffled if x not in ii_id]
    # stop 
    if len(i_pos_val) >= n_pos_val:
      break
  diff_exp_vs_curr = len(i_pos_val) - n_pos_val
  # Training indices is the difference of all vs the validation ones
  i_pos_trn = [x for x in indices_id_shuffled if x not in i_pos_val]
  print(f"Positive Validation PPT, wanted: {n_pos_val}, final: {len(i_pos_val)}")

common_ids = list(set(i_pos_trn).intersection(i_pos_val))
assert not common_ids # should be empty

list_files_pos_val = [filename for i, filename in enumerate(list_files_pos) if i in i_pos_val]
list_files_pos_trn = [filename for i, filename in enumerate(list_files_pos) if i in i_pos_trn]

print("Training: ", len(list_files_pos_trn))
print("Validation: ", len(list_files_pos_val))

common_ids = list(set(list_files_pos_val).intersection(list_files_pos_trn))
print(list_files_pos_val)


# Prepare custom dataset into the detectron2's standard format

In [ ]:
img_dir_json = "/gdrive/MyDrive/AnasPingPong/data/detection/positive_w_bbox"
img_dir_jpeg = "/gdrive/MyDrive/AnasPingPong/data/classification/train_validation/positive"

import glob
from detectron2.structures import BoxMode
import random

def get_pingpong_dicts(d):
    print(img_dir_jpeg)
    print(img_dir_json)
    dataset_dicts = []
 
    if d == 'train':
      json_files_set = list_files_pos_trn
    else:
      json_files_set = list_files_pos_val
    
    count_missing = 0
    for json_file_name in json_files_set:

      json_file = os.path.join(img_dir_json, json_file_name)
      with open(json_file) as f:
        v = json.load(f)
        record = {}
        height = v["imageHeight"]
        width = v["imageWidth"]

        image_filename = os.path.join(img_dir_jpeg, v["imagePath"])

        if not os.path.exists(image_filename):
          count_missing += 1
          continue
        
        record["file_name"] = image_filename
        record["image_id"] = image_filename
        record["height"] = height
        record["width"] = width

        annos = v["shapes"]
        objs = []
        for anno in annos:
            points = anno['points']
            xs = [points[0][0], points[1][0]]
            ys = [points[0][1], points[1][1]]
            x1 = min(xs)
            x2 = max(xs)
            y1 = min(ys)
            y2 = max(ys)

            x_poly = [x1, x2, x2, x1]
            y_poly = [y1, y1, y2, y2]

            poly = [p for x in zip(x_poly, y_poly) for p in x]
            

            obj = {
                "bbox": [x1, y1, x2, y2],
                "bbox_mode": BoxMode.XYXY_ABS,
               # "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    print(f'{d} dataset has {len(dataset_dicts)} samples')
    print(f'missing files: {count_missing}')

    return dataset_dicts


DatasetCatalog.clear()
for d in ["train", "val"]:
    DatasetCatalog.register("ppt_" + d, lambda d=d: get_pingpong_dicts(d))
    MetadataCatalog.get("ppt_" + d).set(thing_classes=["ppt"])
ppt_metadata = MetadataCatalog.get("ppt_train")

#### Check dataloader 
* Visualize some training images and corresponding bounding box 

In [ ]:
dataset_dicts = get_pingpong_dicts("train")
for d in dataset_dicts[:10]:
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=ppt_metadata, scale=2.0)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

# Train

In [ ]:
# check previous models
!ls "/gdrive/MyDrive/AnasPingPong/models/object_detection/"

### Helper function for generating config

In [ ]:
def get_train_cfg(output_dir, config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, max_iter, chekpoint_period, eval_period, flag_resume):
  
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_file_path))
    if flag_resume:
       cfg.MODEL_WEIGHTS = checkpoint_url
    else:
      cfg.MODEL_WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
    cfg.DATASETS.TRAIN = (train_dataset_name,)
    cfg.DATASETS.TEST = (test_dataset_name,)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = max_iter
    cfg.SOLVER.CHECKPOINT_PERIOD = chekpoint_period
    cfg.SOLVER.STEPS = []

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.OUTPUT_DIR = output_dir
    cfg.TEST.EVAL_PERIOD = eval_period

    return cfg

#### Build Evaluator-Trainer class
* Allows evaluating performance on validation set at fixed intervals while training.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

class EvaluateTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
        os.makedirs(os.path.join(cfg.OUTPUT_DIR, "inference"), exist_ok=True)
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


## Start training

In [ ]:
MODEL = "faster_rcnn_R_101_FPN_3x.yaml" # faster_rcnn_R_101_FPN_3x.yaml, faster_rcnn_X_101_32x8d_FPN_3x.yaml
output_dir_basis ="/gdrive/MyDrive/AnasPingPong/models/object_detection/"
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")
output_foldername = "_".join([timestamp, MODEL.split(".")[0]])
output_dir = os.path.join(output_dir_basis, output_foldername)
print(output_dir)

In [ ]:
flag_resume=False

cfg = get_train_cfg(
    output_dir=output_dir,
    config_file_path=os.path.join("COCO-Detection", MODEL),
    checkpoint_url=os.path.join("COCO-Detection", MODEL),
    train_dataset_name="ppt_train",
    test_dataset_name ="ppt_val",
    max_iter=5_000,
    chekpoint_period=len(list_files_pos_trn)*4,
    eval_period=len(list_files_pos_trn)/2,
    flag_resume=flag_resume
)
# from pprint import pprint
# pprint(cfg)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = EvaluateTrainer(cfg) # use DefaultTrainer if you don't need evaluation
trainer.resume_or_load(resume=flag_resume) 
trainer.train()

## Continue training from a checkpoint

* We just beed to load the weights from the last checkpoint and then increase MAX_ITER to the wanted number of training epochs 
* Also set resume=True in trainer.resume_or_load(resume=False)


In [ ]:
# check previous models
!ls "/gdrive/MyDrive/AnasPingPong/models/object_detection/"

In [ ]:
!ls "/gdrive/MyDrive/AnasPingPong/models/object_detection/202112191542_faster_rcnn_R_101_FPN_3x"


In [ ]:
MODEL = "faster_rcnn_R_101_FPN_3x.yaml" # faster_rcnn_R_101_FPN_3x.yaml, faster_rcnn_X_101_32x8d_FPN_3x.yaml
output_dir_basis ="/gdrive/MyDrive/AnasPingPong/models/object_detection/"
timestamp = "202112172206"
output_foldername = "_".join([timestamp, MODEL.split(".")[0]])
output_dir = os.path.join(output_dir_basis, output_foldername)
checkpoint_of_interest = "model_final.pth" # "model_final.pth"
print(os.path.join("COCO-Detection", MODEL))
print(os.path.join(output_dir, "model_final.pth"))


In [ ]:
# output_dir -> Either use one defined in previous cell or select one from list of models
# MODEL -> similar to previous cell or select one from list of models

flag_resume=True

cfg = get_train_cfg(
    output_dir=output_dir,
    config_file_path=os.path.join("COCO-Detection", MODEL),
    checkpoint_url=os.path.join(output_dir, "model_final.pth"),
    train_dataset_name="ppt_train",
    test_dataset_name ="ppt_val",
    max_iter=20_000,
    chekpoint_period=len(list_files_pos_trn)*4,
    eval_period=len(list_files_pos_trn)/2,
    flag_resume=flag_resume

)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = EvaluateTrainer(cfg)
trainer.resume_or_load(resume=flag_resume) 
trainer.train()

#  (TO FIX) Check training curves with Tensorboard

In [ ]:
# Look at training curves in tensorboard:
#!pip3 uninstall tensorboard -y
#!pip3 uninstall tensorflow -y
#!pip3 install --ignore-installed tf-nightly
%load_ext tensorboard
import tensorflow as tf
import datetime, os

%tensorboard --logdir output

#Inference & evaluation using the trained model

### Create predictor

In [ ]:
# check previous models
!ls "/gdrive/MyDrive/AnasPingPong/models/object_detection/"


In [ ]:
!ls "/gdrive/MyDrive/AnasPingPong/models/object_detection/202112172206_faster_rcnn_R_101_FPN_3x"

In [ ]:
MODEL = "faster_rcnn_R_101_FPN_3x.yaml" # faster_rcnn_R_101_FPN_3x.yaml, faster_rcnn_X_101_32x8d_FPN_3x.yaml
output_dir_basis ="/gdrive/MyDrive/AnasPingPong/models/object_detection/"
timestamp = "202112172206" # eg "202112170002" or "202112172206"
output_foldername = "_".join([timestamp, MODEL.split(".")[0]])
output_dir = os.path.join(output_dir_basis, output_foldername)
print(output_dir)
checkpoint_of_interest = "model_final.pth" # "model_final.pth"


In [ ]:
# output_dir -> Either use one define in previous cell or select one from list of models
# MODEL -> similar to previous cell or select one from list of models

flag_resume=True # irrelevant

cfg = get_train_cfg(
    output_dir=output_dir,
    config_file_path=os.path.join("COCO-Detection", MODEL),
    checkpoint_url=os.path.join(output_dir,checkpoint_of_interest),
    train_dataset_name="ppt_train", # irrelevant
    test_dataset_name ="ppt_val", # irrelevant
    max_iter=15_000, # irrelevant
    chekpoint_period=len(list_files_pos_trn)*5, # irrelevant
    eval_period=len(list_files_pos_trn), # irrelevant
    flag_resume=flag_resume # irrelevant

)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, checkpoint_of_interest)  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Check model's predictions on validation data

* *randomly* select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_pingpong_dicts("valid")
for d in dataset_dicts[:10]:
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  print(outputs)
  v = Visualizer(im[:, :, ::-1],
                  metadata=ppt_metadata, 
                  scale=2.0, 
                  instance_mode=ColorMode.IMAGE   
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

# Check model's predictions on test data

In [ ]:
import glob
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a custom testing threshold
predictor = DefaultPredictor(cfg)
files = glob.glob("/gdrive/MyDrive/AnasPingPong/data/classification/test/negative/*")

# files_to_check = ["563529_344025_20.jpeg", "563530_3440248_20.jpeg", "563528_344026_20.jpeg"]

for file in files:
  im = cv2.imread(file)
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  print(outputs)
  v = Visualizer(im[:, :, ::-1],
                  metadata=ppt_metadata, 
                  scale=2.5, 
                  instance_mode=ColorMode.IMAGE   
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

# Model's performance evaluation using COCO's AP metric


In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

def evaluate_on_validation(cfg, thr, data_validation_name):
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = thr
  predictor = DefaultPredictor(cfg)
  evaluator = COCOEvaluator(data_validation_name, output_dir="./output")
  val_loader = build_detection_test_loader(cfg, data_validation_name )
  result = inference_on_dataset(predictor.model, val_loader, evaluator)
  return result

# Check for a threshold of 0.5
result = evaluate_on_validation(cfg, 0.5, "ppt_val")

## For selected model:  Extract AP metrics for different thesholds 

In [ ]:
from pprint import pprint
pprint(cfg)

In [ ]:
# Define thresholds (higher resolution for > 0.9)
thresholds_05_09 = np.arange(0.5, 0.99, 0.01)
thresholds_09_1 = np.arange(0.99, 1.0, 0.001)
thresholds_classification = list(np.concatenate([thresholds_05_09, thresholds_09_1]))

# Estimate metrics for different thresholds
dict_metrics_per_thr = {}
for thr in thresholds_classification:
    print(f"Evaluating performance of model for classification threshold :{thr}")
    metrics = evaluate_on_validation(cfg, np.float(thr), "ppt_val")
    dict_metrics_per_thr[thr] = metrics

# Save dictionary with metrics
filename_metrics_per_threshold = os.path.join(output_dir,str(checkpoint_of_interest.split(".")[0] + "_metrics_diff_thresh.json"))
with open(filename_metrics_per_threshold, 'w') as f_out:
    json.dump(dict_metrics_per_thr, f_out)
